In [38]:
import os
import cv2

# tf tools
import tensorflow as tf

# image processsing
from tensorflow.keras.preprocessing.image import (load_img,
                                                  img_to_array,
                                                  ImageDataGenerator)
# VGG16 model
from tensorflow.keras.applications.vgg16 import (preprocess_input,
                                                 decode_predictions,
                                                 VGG16)
# cifar10 data - 32x32
from tensorflow.keras.datasets import cifar10

# layers
from tensorflow.keras.layers import (Flatten, 
                                     Dense, 
                                     Dropout, 
                                     BatchNormalization)
# generic model object
from tensorflow.keras.models import Model

# optimizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD, Adam

#scikit-learn
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

# for plotting
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path



In [35]:
main_folder_path = Path(("../in/Tobacco3482-jpg"))

PosixPath('../in/Tobacco3482-jpg')

In [42]:


main_folder_path = ("../in/Tobacco3482-jpg") # the folder that contains the images
sorted_dir = sorted(os.listdir(main_folder_path))

images = []
labels = []
for folder in sorted_dir:
    label = folder.split("-")[-1] # extract label from folder name
    folder_path = os.path.join(main_folder_path, folder)
    filenames = sorted(os.listdir(folder_path))
    
    for image in filenames:
        if image.endswith(".jpg"):
            image_path = os.path.join(folder_path, image)
            labels.append(label)
            image = load_img(image_path, target_size=(224, 224))
            image = img_to_array(image)
            image = preprocess_input(image)
            images.append(image)

# Now you have images and corresponding labels
print("Number of images:", len(images))
print("Number of labels:", len(labels))

Number of images: 3482
Number of labels: 3482


In [28]:
images

[array([[[[-103.939  , -116.779  , -123.68   ],
          [-103.939  , -116.779  , -123.68   ],
          [-103.939  , -116.779  , -123.68   ],
          ...,
          [ 151.061  ,  138.22101,  131.32   ],
          [ 151.061  ,  138.22101,  131.32   ],
          [ 151.061  ,  138.22101,  131.32   ]],
 
         [[ 151.061  ,  138.22101,  131.32   ],
          [ 151.061  ,  138.22101,  131.32   ],
          [ 151.061  ,  138.22101,  131.32   ],
          ...,
          [ 151.061  ,  138.22101,  131.32   ],
          [ 151.061  ,  138.22101,  131.32   ],
          [ 151.061  ,  138.22101,  131.32   ]],
 
         [[ 151.061  ,  138.22101,  131.32   ],
          [ 151.061  ,  138.22101,  131.32   ],
          [ 151.061  ,  138.22101,  131.32   ],
          ...,
          [ 151.061  ,  138.22101,  131.32   ],
          [ 151.061  ,  138.22101,  131.32   ],
          [ 151.061  ,  138.22101,  131.32   ]],
 
         ...,
 
         [[ 151.061  ,  138.22101,  131.32   ],
          [ 151.06

In [43]:
labels

['ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 'ADVE',
 

In [24]:
X_trains = []
for image in images:
    X_train = image
    X_train = X_train / 255
    X_trains.append(X_train)

In [5]:
labelNames = ["ADVE", "Email", 
              "Form", "Letter", 
              "Memo", "News", 
              "Note", "Report", 
              "Resume", "Scientific"]

In [30]:
X_train.shape

(1, 224, 224, 3)

In [31]:
model = VGG16(include_top=False, 
              pooling='avg',
              input_shape=(224, 224, 3))